In [7]:
import numpy as np
import pandas as pd
import pickle
from genetic_selection import GeneticSelectionCV
from sklearn.ensemble import RandomForestClassifier


import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
train.head()

,id,ptt_max,ptt_min,inr_min,inr_max,pt_min,pt_max,calcium_max.1,calcium_min.1,temperature_max,temperature_min,temperature_mean,glucose_min,glucose_max,glucose_mean,weight_admit,glucose_max.2,glucose_min.2,aniongap_max,aniongap_min,gcs_motor,potassium_min.1,potassium_max.1,bicarbonate_max.1,bicarbonate_min.1,sodium_min.1,sodium_max.1,chloride_min.1,chloride_max.1,gcs_verbal,hemoglobin_max.1,hemoglobin_min.1,wbc_max,wbc_min,platelets_max,platelets_min,hematocrit_max.1,hematocrit_min.1,bun_max,bun_min,gcs_eyes,gcs_min,gcs_unable,resp_rate_mean,resp_rate_max,resp_rate_min,spo2_max,spo2_mean,spo2_min,mbp_min,mbp_max,mbp_mean,dbp_max,dbp_min,sbp_mean,sbp_max,sbp_min,dbp_mean,heart_rate_min,heart_rate_max,heart_rate_mean,race,gender,admission_age,aki
0,36570066,29.5,28.6,1.1,1.1,11.9,12.0,9.0,8.1,36.39,35.89,36.222000,76.0,211.0,146.2,110.0,167.0,76.0,20.0,15.0,6.00,4.8,6.7,28.0,23.0,131.0,138.0,93.0,101.0,5.0,9.8,9.2,14.3,11.0,333.0,304.0,29.2,27.8,52.0,47.0,4.0,15.0,0.0,16.833333,24.0,13.0,100.0,98.916667,96.0,56.0,69.0,63.090909,58.0,40.0,116.136364,126.0,103.0,47.863636,96.0,104.0,100.083333,7,0,79.953141,3
1,39307659,150.0,28.4,1.9,2.3,20.0,24.7,8.4,7.9,37.00,36.28,36.558000,127.0,132.0,129.5,82.0,143.0,127.0,22.0,13.0,6.00,2.9,4.5,29.0,19.0,126.0,132.0,89.0,93.0,5.0,13.0,12.1,14.9,12.0,247.0,199.0,39.5,36.3,30.0,23.0,4.0,15.0,0.0,29.611111,38.0,17.0,98.0,95.444444,91.0,65.0,96.0,79.555556,89.0,56.0,109.833333,127.0,97.0,70.166667,72.0,134.0,97.263158,32,0,78.194169,0
2,38743306,26.0,26.0,1.1,1.1,12.1,12.1,8.7,8.7,37.00,36.67,36.805000,207.0,305.0,254.0,62.1,266.0,266.0,12.0,12.0,6.00,3.8,3.8,27.0,27.0,137.0,137.0,102.0,102.0,5.0,9.5,9.5,7.5,7.5,346.0,346.0,31.1,31.1,8.0,8.0,4.0,15.0,0.0,16.000000,19.0,14.0,100.0,97.166667,94.0,69.0,108.0,85.692308,99.0,56.0,112.153846,143.0,95.0,73.307692,60.0,97.0,84.166667,28,0,65.602396,2
3,32339865,26.1,26.1,1.2,1.2,12.8,12.8,9.5,8.3,37.28,36.83,37.087143,111.0,120.0,117.0,113.1,136.0,120.0,11.0,10.0,1.00,4.3,4.5,39.0,36.0,140.0,141.0,92.0,93.0,0.0,18.6,16.1,7.2,5.1,178.0,146.0,62.8,56.5,39.0,39.0,1.0,15.0,1.0,22.518519,31.0,14.0,100.0,94.800000,89.0,71.0,109.0,96.720000,94.0,60.0,138.160000,150.0,113.0,80.200000,59.0,87.0,71.461538,27,0,64.906629,2
4,35526987,39.4,27.7,1.0,1.1,10.4,11.9,8.0,7.5,38.33,36.78,37.158750,158.0,213.0,187.0,97.4,196.0,132.0,16.0,13.0,1.61,4.2,4.5,22.0,18.0,137.0,137.0,103.0,109.0,0.0,17.1,14.0,21.1,14.6,208.0,147.0,54.3,44.0,23.0,22.0,1.0,15.0,1.0,24.230159,32.0,18.0,99.0,95.951613,90.0,56.0,100.0,70.475806,95.0,47.0,97.672131,127.0,81.0,61.827869,57.0,100.0,82.387097,28,1,57.438861,2


特征选择策略：
1. 数据处理过程中首先删除掉了缺失比例大于 30% 的特征，这部分特征缺失值太多，通过填补的方式得到的数据集反而可能引入更大的噪音。
2. 使用遗传算法进行特征选择

遗传算法的原理：
遗传算法是一种启发式算法，它使用一种基于进化的方法来确定最优集，基本原理是仿效生物界中的“物竞天择，适者生存”的演化法则。
遗传算法首先要有一个代表目标问题的种群，这个种群是由经过基因编码的一定数目的个体组成。我们的目标就是通过遗传算法通过这个初始种群不断的对个体的基因编码做调整逐渐寻找到最优或者次优的个体。首先实现从表现型到基因型的编码，然后按照设置的最大基因数量随机做基因编码排序生成一定数目的个体组成初代种群，然后就开始逐代演化。在每一代中根据个体的适应度（fitness）的大小去做选择，摘出其中表现比较好的个体然后借助自然遗传学的遗传因子（genetic operators）进行组合交叉和变异，产生出代表新的解集的种群。这就像种群的自然进化一样，新的种群一定是向着我们设定的适应度去演化的，后代种群会更适应环境。末代种群中的最优个体可以作为问题的近似最优解。

对于特征选择，我们的目标是从当前 61 个特征中筛选出对于标签影响更大的特征（比如 30 个），步骤如下：
1. 用随机生成的个体初始化种群
    这里的 61 个特征就可以理解为遗传算法中的基因，我们从中随机挑选出 30 个就组成一个个体（特征子集），按照设定的初始种群个数随机生成个体组成初代种群。
2. 选择一个模型去对每个个体做训练并使用选择的评估指标评估该个体的适应度
3. 去除适应性差的个体（选择）。
4. 构建新的个体（交叉）。
    将种群中的各个个体随机搭配成对，对每一个个体以交叉概率交换它们的部分特征，得到新一代个体。
5. 恢复丢失的特征（突变）。
    对种群中的每个个体，以变异概率改变某个或多个特征为其他特征，保证整个解空间的多样性。
6. 重复步骤 2-5，直到满足停止条件（例如，可以是迭代次数）。
    该算法运行一定数量的代之后，群体的最优成员就是选定的特征。


初始种群数量如果比较小，则可能导致需要很长的时间才能遍历整个解空间；如果比较大则可能出现2个个体很相似，导致我们的计算重复，计算的规模比较大。

使用 sklearn-genetic 包中的 GeneticSelectionCV 算法做具体的处理，选用随机森林作为模型，准确率作为评估个体性能的适应度函数。


In [4]:
y_train = train['aki'].astype(float)
x_train = train.drop(['aki'], axis=1)

In [5]:
model = RandomForestClassifier(n_estimators=100)

selection = GeneticSelectionCV(model,
                              cv=5,
                              scoring="accuracy",
                              max_features=30,
                              n_population=120,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=50,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              n_gen_no_change=10,
                              n_jobs=-1)

selection = selection.fit(x_train, y_train)
print(selection.support_)


[False False  True False  True False False False  True False False  True
  True False  True  True False False  True  True False  True False  True
 False False False False  True  True False  True False False False False
 False  True  True False  True False False False  True  True False  True
  True False False False False  True  True False  True False False  True
  True  True False  True]


In [8]:
with open('genetic_selection_result.pkl', 'wb') as file:
    pickle.dump(selection, file)

In [12]:
selected_features = selection.get_support()
column_names = list(x_train.columns)
selected_feature_names = []
print("Selected Features:")
for ind, feature in enumerate(selected_features):
    if feature:
        print(f"{ind} {column_names[ind]}")
        selected_feature_names.append(column_names[ind])


Selected Features:
2 ptt_min
4 inr_max
8 calcium_min.1
11 temperature_mean
12 glucose_min
14 glucose_mean
15 weight_admit
18 aniongap_max
19 aniongap_min
21 potassium_min.1
23 bicarbonate_max.1
28 chloride_max.1
29 gcs_verbal
31 hemoglobin_min.1
37 hematocrit_min.1
38 bun_max
40 gcs_eyes
44 resp_rate_max
45 resp_rate_min
47 spo2_mean
48 spo2_min
53 dbp_min
54 sbp_mean
56 sbp_min
59 heart_rate_max
60 heart_rate_mean
61 race
63 admission_age


遗传算法选择的结果为：

['ptt_min','inr_max','calcium_min.1','temperature_mean','glucose_min','glucose_mean','weight_admit','aniongap_max','aniongap_min','potassium_min.1','bicarbonate_max.1','chloride_max.1','gcs_verbal','hemoglobin_min.1','hematocrit_min.1','bun_max','gcs_eyes','resp_rate_max','resp_rate_min','spo2_mean','spo2_min','dbp_min','sbp_mean','sbp_min','heart_rate_max','heart_rate_mean','race','admission_age']

共 28 个特征